<a href="https://colab.research.google.com/github/underwater84/Shelter.Animal.Outcome/blob/master/market_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install scikit-learn scipy numpy tslearn==0.5.3.2 yfinance plotly kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.6 MB/s eta 0:00:00


In [12]:
from datetime import date,datetime,timedelta
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import StandardScaler
import yfinance as yf
import pandas as pd
from pandas_datareader import data
import plotly.graph_objects as go
import plotly.express as px
import kaleido
import requests
pd.options.plotting.backend = "plotly"

In [18]:

def check_cluster(df,title,col,state_col):#graph_path):
    current_state = df['sorted_state'].values[-1]
    fig = px.scatter(df[[col,state_col]], color=state_col,title = title+ 'last_stat_'+str(current_state),log_y=True)
    fig.show(renderer="colab")
    # img = fig.to_image(format='jpg')
    # url = 'https://api.telegram.org/bot{}/sendPhoto?chat_id={}'.format('1812047127:AAFDK2nXQVtxayq22Ktw_2gM6kbxfZUZEG4','-582799890')
    # requests.post(url, files={'photo': img})
    # url = 'https://api.telegram.org/bot{}/sendPhoto?chat_id={}'.format('1933482882:AAGCdloVkJPx0iVRjDAuIBMby9wktxqiNgk','-564893609')
    # requests.post(url, files={'photo': img})


def to_cluster2(df_train,df_predict,n_cluster):
    km = TimeSeriesKMeans(n_clusters=n_cluster, verbose=True, random_state=123)
    km.fit(df_train[:'2021-01-01'].values)
    y_pred = km.predict(df_predict.values)
    return y_pred, km.inertia_



def standardscaler_scaleer(full_df,col):
    scaler = StandardScaler()
    scaler.fit(full_df[[col]])
    values_2_return = scaler.transform(full_df[[col]])
    return values_2_return



def sort_state(df):
    state_map = {}
    all_states = df['state'].unique()
    for state in all_states:
        state_map[state] = df[df['state'] == state]['btc_pct_change'].mean()
    sorted_dict = dict(sorted(state_map.items(), key=lambda item: item[1]))
    df['sorted_state'] = 0
    leger_dict = {}
    sorted_keys_list = list(sorted_dict.keys())
    for i in range(len(sorted_keys_list)):
        leger_dict[i] = sorted_keys_list[i]
    for i in leger_dict:
        df.loc[df['state'] == leger_dict[i],'sorted_state'] = i
    return df



def get_current_state(df,title):
    df[str(1) + '_pct_change'] = df['Adj Close'].pct_change(1)
    df[str(2) + '_pct_change'] = df['Adj Close'].pct_change(2)
    df[str(4) + '_pct_change'] = df['Adj Close'].pct_change(4)
    df[str(8) + '_pct_change'] = df['Adj Close'].pct_change(8)
    df[str(16) + '_pct_change'] = df['Adj Close'].pct_change(16)
    # df[str(32) + '_pct_change'] = df['Adj Close'].pct_change(32)
    df['2nd_dev_1_day'] = df['Adj Close'].diff().diff()
    df['2nd_dev_4_day'] = df['Adj Close'].diff().diff(4)
    df['2nd_dev_8_day'] = df['Adj Close'].diff().diff(8)
    df['2nd_dev_16_day'] = df['Adj Close'].diff().diff(16)

    cols_4_scaler = [col for col in df.columns if 'pct' in col]  # or '2nd' in col]

    for col in cols_4_scaler:
        df['robust_' + col] = standardscaler_scaleer(df, col)
        div_val = df['robust_' + col].max() / 2
        df['robust_' + col] = df['robust_' + col] / div_val

    cols_2_dave = [col for col in df.columns if 'robust' in col]
    btc_df_no_na = df.dropna()

    n_cluster_dis_results = {}

    btc_df_no_na['btc'] = btc_df_no_na['Adj Close']
    n_class = 3
    btc_df_no_na['state'], n_cluster_dis_results[n_class] = to_cluster2(btc_df_no_na[cols_2_dave],
                                                                        btc_df_no_na[cols_2_dave], n_class)
    btc_df_no_na['btc_pct_change'] = btc_df_no_na['Adj Close'].pct_change()
    # btc_df_no_na
    btc_df_sorted = sort_state(btc_df_no_na)
    check_cluster(btc_df_sorted[['btc', 'sorted_state']]['2021-01-01':],
                  '{}_graph_with_{}_classes_'.format(title,n_class),
                  'btc', 'sorted_state')#, graph_path)






In [20]:
# if __name__ == "__main__":
    #get data
today = date.today()
today_d = today.strftime("%Y-%m-%d")
tomorrow = today + timedelta(days=1)
d1 = tomorrow.strftime("%Y-%m-%d")
start_date = datetime(2018, 10, 19)
start_date = start_date.strftime("%Y-%m-%d")
# btc_df = yf.download("BTC-USD", start=start_date,threads=False) #, end=today_d
# eth_df = yf.download("ETH-USD", start=start_date, end=d1)

start_date = '2010-01-01'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
eth_df = yf.download('ETH-USD', start_date)
btc_df = yf.download('BTC-USD', start_date)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [9]:
eth_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09 00:00:00+00:00,308.644989,329.451996,307.056000,320.884003,320.884003,893249984
2017-11-10 00:00:00+00:00,320.670990,324.717987,294.541992,299.252991,299.252991,885985984
2017-11-11 00:00:00+00:00,298.585999,319.453003,298.191986,314.681000,314.681000,842300992
2017-11-12 00:00:00+00:00,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936
2017-11-13 00:00:00+00:00,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984
...,...,...,...,...,...,...
2023-01-31 00:00:00+00:00,1567.419922,1598.517578,1563.804321,1586.535400,1586.535400,6585191019
2023-02-01 00:00:00+00:00,1586.488159,1644.727539,1566.857300,1641.792725,1641.792725,8116969489
2023-02-02 00:00:00+00:00,1641.365967,1704.458130,1641.322632,1643.241577,1643.241577,10558081069


In [19]:
get_current_state(eth_df,'ETH')

<ipython-input-18-1c4b81933660>:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1.992 --> 1.581 --> 1.533 --> 1.513 --> 1.496 --> 1.487 --> 1.481 --> 1.477 --> 1.474 --> 1.473 --> 1.473 --> 1.472 --> 1.472 --> 1.471 --> 1.471 --> 1.471 --> 1.471 --> 1.471 --> 


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:931: FutureWarning:

Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.

/usr/local/lib/python3.8/dist-packages/tslearn/utils/utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 1897 1-dimensional timeseries

<ipython-input-18-1c4b81933660>:71: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-1c4b81933660>:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [21]:
get_current_state(btc_df,'BTC')

<ipython-input-18-1c4b81933660>:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.321 --> 0.252 --> 0.226 --> 0.215 --> 0.211 --> 0.210 --> 0.209 --> 0.209 --> 0.209 --> 0.209 --> 0.209 --> 0.209 --> 0.209 --> 0.209 --> 


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:931: FutureWarning:

Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.

/usr/local/lib/python3.8/dist-packages/tslearn/utils/utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 3046 1-dimensional timeseries

<ipython-input-18-1c4b81933660>:71: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-1c4b81933660>:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-